In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pandas
import glob
import xml.etree.ElementTree as ET
import nltk
import math
import numpy
import warnings

warnings.filterwarnings("ignore")

In [2]:
atos_dict = {
    
"Ato_Retificacao_Comissionado" : ['tipo_documento','data_documento','numero_dodf','data_dodf','pagina_dodf','tipo_ato',
'nome','lotacao','informacao_errada','informacao_corrigida','tipo_edicao', 'Ato_Retificacao_Comissionado'],

"Ato_Tornado_Sem_Efeito_Exo_Nom" : ['tipo_documento','data_documento','tipo_edicao','numero_dodf','data_dodf','pagina_dodf',
'nome','cargo_efetivo','matricula','matricula_SIAPE','simbolo','cargo_comissionado','hierarquia_lotacao','orgao', 'Ato_Tornado_Sem_Efeito_Exo_Nom'],
    
"Ato_Exoneracao_Comissionado" : ['nome','matricula','simbolo','cargo_comissionado','hierarquia_lotacao','orgao','vigencia','a_pedido_ou_nao',
'cargo_efetivo','matricula_SIAPE','motivo', 'Ato_Exoneracao_Comissionado'],
    
"Ato_Nomeacao_Comissionado": ['nome', 'cargo_efetivo', 'matricula', 'matricula_SIAPE', 'simbolo', 'cargo_comissionado',
'hierarquia_lotacao', 'orgao', 'Ato_Nomeacao_Comissionado'],
    
"Ato_Nomeacao_Efetivo": ['edital_normativo','data_edital_normativo','numero_dodf_edital_normativo','data_dodf_edital_normativo',
'edital_resultado_final','data_edital_resultado_final','numero_dodf_resultado_final','data_dodf_resultado_final','cargo',
'especialidade','carreira','orgao','candidato','candidato_PNE','processo_SEI', 'Ato_Nomeacao_Efetivo'],

"Ato_Abono_Permanencia" : ['nome','matricula','cargo_efetivo','classe','padrao','quadro','fundamento_legal',
'orgao','processo_SEI','vigencia','matricula_SIAPE', 'Ato_Abono_Permanencia'],    
    
"Ato_Substituicao" : ['nome_substituto','matricula_substituto','nome_substituido','matricula_substituido','cargo_substituto',
'simbolo_substituto','cargo_objeto_substituicao','simbolo_objeto_substituicao','hierarquia_lotacao','orgao','data_inicial',
'data_final','matricula_SIAPE','motivo', 'Ato_Substituicao'],
    
"Ato_Tornado_Sem_Efeito_Apo" : ['tipo_documento','numero_documento','fundamento_legal', 'data_documento','numero_dodf','data_dodf','pagina_dodf','nome',
'matricula','matricula_SIAPE','cargo_efetivo','classe','padrao','quadro','orgao','processo_SEI', 'Ato_Tornado_Sem_Efeito_Apo'],

"Ato_Retificacao_Efetivo" : ['tipo_documento','numero_documento','data_documento','numero_dodf','data_dodf','pagina_dodf',
'tipo_ato','nome','matricula','cargo_efetivo','classe','matricula_SIAPE','padrao','lotacao','informacao_errada','informacao_corrigida',
'tipo_edicao', 'Ato_Retificacao_Efetivo'],

"Ato_Cessao" : ['nome','matricula', 'cargo_efetivo','classe','padrao','orgao_cedente','orgao_cessionario','onus','fundamento_legal',
'processo_SEI','vigencia','matricula_SIAPE','cargo_orgao_cessionario','simbolo','hierarquia_lotacao', 'Ato_Cessao'],

"Ato_Reversao" : ['nome','matricula', 'motivo','cargo_efetivo','classe','padrao','quadro','fundamento_legal','orgao',
'processo_SEI','vigencia','matriucla_SIAPE', 'Ato_Reversao'],

"Ato_Exoneracao_Efetivo" : ['nome','matricula','cargo_efetivo','classe','padrao','carreira','quadro','orgao','processo_SEI','vigencia','matricula_SIAPE',
'motivo','fundamento_legal', 'Ato_Exoneracao_Efetivo'],

}

---
# 1. Extraindo trechos anotados dos XMLs

### 1.1. Adiquirindo a raiz de cada XML

In [3]:
glob_path = 'xml_batch1/*.xml' # Caminho até os XMLs
roots = []
for xml in glob.glob(glob_path):
    tree = ET.parse(xml)
    roots.append(tree.getroot())

### 1.2. Iremos agora iterar sobre a raiz de cada arquivo buscando pelas relações contidas nelas. Em seguida iremos usar as relações para compor cada linha do CSV.
Antes é importante entender a estrutura XML da anotação de uma relação:
```xml
<relation id="R1">
    <infon key="type">Ato_Tornado_Sem_Efeito_Exo_Nom</infon>
    <infon key="annotator">tatiana_franco</infon>
    <infon key="updated_at">2020-12-22T02:09:50Z</infon>
    <node refid="4192" role=""/>
    <node refid="4193" role=""/>
    <node refid="4194" role=""/>
    <node refid="4195" role=""/>
    <node refid="4196" role=""/>
    <node refid="4197" role=""/>
    <node refid="4198" role=""/>
    <node refid="4199" role=""/>
    <node refid="118512" role=""/>
    <node refid="118513" role=""/>
    <node refid="118514" role=""/>
</relation>
```
> Cada nó(node) representa um anotação. Basta recuperar o *refid* de cada nó e procurar pelo ID de sua respectiva anotação.

> Utilize a tag *infon* com *key=type* para identificar o ato anotado

```xml
<annotation id="4192">
    <infon key="type">tipo_documento</infon>
    <infon key="identifier"></infon>
    <infon key="annotator">lindeberg</infon>
    <infon key="updated_at">2020-12-22T00:41:28Z</infon>
    <location offset="385761" length="7"/>
    <text>Decreto</text>
</annotation>
```
> Novamente utilize *infon* com *key=type* para identificar qual o tipo de anotação e a tag *text* para resgatar o texto da anotação


In [4]:
# Esse bloco deve demorar de alguns segundos a 1 minuto
atos_csv_dict = {}

for root in roots: # Intera sobre as raizes
    for relation in root.findall(".//relation"):                            # Intera sobre as relações.
        row_act = {}
        type_relation = relation.find('.//infon[@key="type"]').text
        for node in relation.findall('node'):                               # Intera sobre os nós(anotações) da relação.
            ref_id = node.get('refid')
            annotation = root.find(f'.//annotation[@id="{ref_id}"]')        # Encontra anotação.
            type_annotation = annotation.find('.//infon[@key="type"]').text # Encontra tipo da anotação.
            text_annotation = annotation.find('text').text                  # Encontra texto da anotação.
            row_act[type_annotation] = text_annotation
        
        if type_relation not in atos_csv_dict:                              # Checa se a tabela já existe, caso contrário, cria uma.
            atos_csv_dict[type_relation] = pandas.DataFrame()
            
        atos_csv_dict[type_relation] = atos_csv_dict[type_relation].append(row_act, ignore_index=True)

Como resultado final temos um dicionario de CSVs. Cada chave do dicionário é referente a um ato e aponta para seu respectivo dataframe.

---

# 2. Exploração e validação da integridade dos dados

### 2.2. Vamos checar agora quantos atos temos e se os Dataframes estão coerentes.

In [5]:
print(atos_csv_dict.keys())
print("Quantidade de atos: " + str(len(atos_csv_dict.keys())))

dict_keys(['Ato_Nomeacao_Comissionado', 'Ato_Exoneracao_Comissionado', 'Ato_Exoneracao_Efetivo', 'Ato_Retificacao_Comissionado', 'Ato_Substituicao', 'Ato_Retificacao_Efetivo', 'Ato_Tornado_Sem_Efeito_Exo_Nom', 'Ato_Cessao', 'Ato_Nomeacao_Efetivo', 'Ato_Abono_Permanencia', 'Ato_Tornado_Sem_Efeito_Apo', 'Ato_Reversao'])
Quantidade de atos: 12


### 2.3. Temos todos os 12 atos presentes no dicionário. Checaremos agora o tamanho e os campos de cada dataframe.

In [6]:
for key in atos_csv_dict.keys():
    print("Ato: " + key)
    print("Campos: ", end='')
    print(atos_csv_dict[key].columns)
    print("Tamanho: " + str(len(atos_csv_dict[key])))
    print('----------------')

Ato: Ato_Nomeacao_Comissionado
Campos: Index(['Ato_Nomeacao_Comissionado', 'cargo_comissionado', 'cargo_efetivo',
       'hierarquia_lotacao', 'matricula', 'nome', 'orgao', 'simbolo',
       'Ato_Retificacao_Comissionado', 'data_documento', 'data_dodf',
       'informacao_corrigida', 'informacao_errada', 'numero_dodf',
       'pagina_dodf', 'tipo_ato', 'tipo_documento', 'Ato_Retificacao_Efetivo',
       'matricula_SIAPE', 'carreira', 'Ato_Exoneracao_Comissionado',
       'Ato_Tornado_Sem_Efeito_Exo_Nom', 'motivo'],
      dtype='object')
Tamanho: 7687
----------------
Ato: Ato_Exoneracao_Comissionado
Campos: Index(['Ato_Exoneracao_Comissionado', 'cargo_comissionado',
       'hierarquia_lotacao', 'nome', 'orgao', 'simbolo', 'motivo', 'matricula',
       'a_pedido_ou_nao', 'cargo_efetivo', 'vigencia', 'matricula_SIAPE',
       'data', 'carreira', 'Ato_Nomeacao_Comissionado'],
      dtype='object')
Tamanho: 755
----------------
Ato: Ato_Exoneracao_Efetivo
Campos: Index(['Ato_Exoneracao_Efe

### 2.4. Extraido as tabelas dos atos precisamos agora executar uma limpeza nas tebelas.

> REALIZANDO TRATAMENTO PARA TODOS OS ATOS 

In [7]:
for key in atos_dict:
    dframe = atos_csv_dict[key]
    fields = atos_dict[key]
    # Nessa linha todas as colunas que não pertencem a nomecao são extraidas para uma segunda lista. (Compreensão de listas).
    non_fields = [column for column in dframe.columns if column not in fields]
    # Exclusão de todas as linhas que possuam algum valor nos campos que não pertecem a nomeacao.
    if (len(non_fields) > 0):
        dframe = dframe[dframe[non_fields].isna().any(axis=1)]
    # Exclusão de todas as colunas que não pertecem a nomeação.
    dframe = dframe.drop(columns=non_fields)
    # Exclusão das linhas que não possuem anotação de atos.
    dframe = dframe.dropna(subset=[key])
    atos_csv_dict[key] = dframe
    #atos_csv_dict[key]

Um problema comum em todos os atos é a existência de valores *nan* no campo de texto do ato. Sem esse campo não é possível executar a etapa de criação de labels. Podemos assim realizar um laço que retirar todas as linhas que possuem esse campo com valor de *nan*.

In [8]:
for key in atos_csv_dict.keys():
    atos_csv_dict[key] = atos_csv_dict[key].dropna(subset=[key])

---
# 3. Criação dos labels IOB

### 3.1. Definiremos agora uma função *generate_labels* que recebe uma linha do csv do ato e adiciona o label de IOB.
> No formato IOB o caracter 'B' representa o início de uma entidade o 'I' uma parte de uma entidade q já tenha sido iniciada e 'O' um token que não possui label.
> Dessa forma adiciona-se o tipo da entidade em seguida do label identificado 'B-entidade'...'I-entidade'

In [9]:
# Altere aqui a forma de tokenizar as sentenças.
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
#     tokenizer = nltk.RegexpTokenizer(r"\w+")
#     new_words = tokenizer.tokenize(sentence)
#     return new_words

def find_entity(row, token):
    for column in row.keys()[1:]:
        if row[column] is not numpy.nan and token == tokenize(row[column])[0]:
            return column
    
    return None

# Atualizar no futuro para qualquer ato.
# Aparentemente esse algoritmo está O(n*m) onde n é a quantidade de tokens e m a quantidade de colunas do df.
def generate_IOB_labels(row, key):
    labels = []
    entity_started = False
    for token in tokenize(row[key]):                         # Intera sobre cada token da anotação do ato.
        if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
            entity = find_entity(row, token)                 # Busca o token atual no primeiro token de todos os campos do df.
            if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                entity_started = True
                token_index = 1
                labels.append('B-' + entity)
            else:
                labels.append('O')
        else:                                                # Caso uma entidade já tenha sido identificada
            if token_index < len(tokenize(row[entity])) and token == tokenize(row[entity])[token_index]: # Checa se o próximo token pertence à entidade e se o tamanho da entidade chegou ao fim.
                labels.append('I-' + entity)                 # Se a entidade ainda possui tokens e a comparação foi bem sucedida adicione o label I.
                token_index += 1
                if token_index >= len(tokenize(row[entity])):
                    entity_started = False
            else:                                            # Se o token n for igual ou a entidade chegou ao fim.
                entity_started = False
                labels.append('O')
                
    return labels


### 3.2. Agora, com as funções de geração label prontas iremos criar uma lista de strings representando os labels para adicionar ao df de cada ato.

In [10]:
for key in atos_csv_dict.keys():
    labels_row = []
    for index, row in atos_csv_dict[key].iterrows():
        try:
            labels_row.append(' '.join(generate_IOB_labels(row, key)))
        except Exception as e:
            print(row)
            raise e
        
    atos_csv_dict[key].insert(len(atos_csv_dict[key].columns), 'labels_IOB', labels_row, False)

---
# 4. Criação das features e treinamento do CRF à moda José

> Faremos duas funções para gerar as features para o algoritmo CRF.

In [11]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq, key):
    x = []
    y = []
    for index, row in arq.iterrows():
        x.append(extract_features(tokenize(row[key])))
        y.append(row['labels_IOB'].split())
    return x, y

> Aqui criaremos os dados usando 70% pra treinamento e 30% teste. Isso é feito para cada ato e ao final é apresentado a tebela do F1-Score

In [12]:
training_ratio = 0.7  

for key in atos_csv_dict.keys():
        print("------------------------------------------------------------------------------------")
        print("Ato:" + key)
        print("Tamanho: " + str(len(atos_csv_dict[key])))
        x, y = separate_cols(atos_csv_dict[key], key)
        train_x = x[:math.floor(training_ratio*len(atos_csv_dict[key]))]
        train_y = y[:math.floor(training_ratio*len(atos_csv_dict[key]))]
        test_x = x[math.floor(training_ratio*len(atos_csv_dict[key])):]
        test_y = y[math.floor(training_ratio*len(atos_csv_dict[key])):]

        len(atos_csv_dict[key]), len(train_x), len(train_y), len(test_x), len(test_y)

        model = sklearn_crfsuite.CRF(
            algorithm = 'l2sgd', 
            c2=1,
            max_iterations=100, 
            all_possible_transitions=True,
            verbose=False
        )

        model.fit(train_x, train_y)
        y_pred = model.predict(test_x)

        labels = list(model.classes_)
        labels.remove('O')

        f1 = metrics.flat_f1_score(test_y, y_pred, 
                              average='weighted', labels=labels)
        print(f1)
        print(metrics.flat_classification_report(
            test_y, y_pred, labels=labels, digits=3
        ))

------------------------------------------------------------------------------------
Ato:Ato_Nomeacao_Comissionado
Tamanho: 619
0.8859061349803052
                      precision    recall  f1-score   support

              B-nome      0.962     0.984     0.973       182
              I-nome      0.980     0.988     0.984       502
     B-cargo_efetivo      1.000     0.200     0.333        25
     I-cargo_efetivo      1.000     0.250     0.400        68
         B-matricula      1.000     1.000     1.000        37
           B-simbolo      0.957     0.989     0.972       178
B-cargo_comissionado      0.883     0.954     0.917       174
I-cargo_comissionado      0.752     0.622     0.680       185
B-hierarquia_lotacao      0.885     0.851     0.868       181
I-hierarquia_lotacao      0.921     0.920     0.920      3163
             B-orgao      0.739     0.748     0.744       159
             I-orgao      0.830     0.787     0.808      1158
         I-matricula      1.000     1.000     